## Install OCR Tool

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.


In [2]:
import cv2
import numpy as np
import random
import pytesseract
import re
from google.colab.patches import cv2_imshow

## Load YOLO Weights
Put the path to your ".weights" file at the first line

In [3]:
weight_file_path = "yolov3_training_last.weights"
net = cv2.dnn.readNet(weight_file_path, "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

## Target Classes

In [4]:
# Name custom object
classes = ["do not enter", "guide", "interstate highway", "us highway", "speed limit", "one way",
           "no left turn", "stop", "no right turn", "no u turn", "railway crossing", "yield"]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

## YOLO Object Detection
Modified from [Sergio Canu - Train YOLO to detect a custom object (online with free GPU)](https://pysource.com/wp-content/uploads/2020/04/train_yolo_to_detect_custom_object.zip)

In [5]:
def yolo(img):
    # Expected size in 600 * 800:
    ratio = min(max(600 / img.shape[0], 800 / img.shape[1]), 1)
    img = cv2.resize(img, None, fx=ratio, fy=ratio)
    height, width, channels = img.shape
    # Detecting objects:
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # Getting classes and bounding boxes:
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:  # object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    # Non-maximum suppression:
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    results = [(class_ids[i], boxes[i]) for i in range(len(boxes)) if i in indexes]
    # Showing results:
    img_toshow = img.copy()
    for class_id, box in results:
        x, y, w, h = box
        color = colors[class_id]
        cv2.rectangle(img_toshow, (x, y), (x + w, y + h), color, 2)
    cv2_imshow(img_toshow)
    return img, results

## Optical Character Recognition
Credit to [Hucker Marius - Optical Character Recognition (OCR) with less than 10 Lines of Code using Python](https://towardsdatascience.com/optical-character-recognition-ocr-with-less-than-12-lines-of-code-using-python-48404218cccb)

In [6]:
def ocr(img, yolo_results):
    for class_id, box in yolo_results:
        x, y, w, h = box
        crop = img[y:y+h, x:x+w, :]
        crop = cv2.resize(crop, None, fx=2, fy=2)
        cv2_imshow(crop)
        # Alternatively: can be skipped if you have a Blackwhite image
        gray = cv2.cvtColor(crop, cv2.COLOR_RGB2GRAY)
        gray, img_bin = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        gray = cv2.bitwise_not(img_bin)
        # Removing noise by morphological operations:
        kernel = np.ones((3, 3), np.uint8)
        crop = cv2.erode(gray, kernel, iterations=1)
        crop = cv2.dilate(crop, kernel, iterations=1)
        cv2_imshow(crop)
        # Recognizing characters:
        output = pytesseract.image_to_string(crop)
        # Optimizing output:
        lines = re.split("\n+", output)
        new_lines = []
        for line in lines:
            words = re.split("\\s+", line.strip().upper())
            new_line = " ".join(words)
            if new_line not in ("", "W", "WO", "OW"):
                new_lines.append(new_line)
        new_output = "\n".join(new_lines)
        print("OUTPUT:\n")
        print(new_output)

## Test the Model on Images
Put the path to your image file at the first line

In [ ]:
image_path = "Test1.jpg"
img = cv2.imread(image_path)
img, yolo_results = yolo(img)
ocr(img, yolo_results)